# SchoolDatabase - Preprocessing

## Introduction

This notebook serves as a preprocessing tool for the SchoolDatabase project, designed to load educational data from CSV files hosted on GitHub into an SQLite database. It aims to establish a clean, organized, and well-documented codebase for loading and preprocessing.

The notebook is structured to perform the following tasks:

1. Establish a database connection to SQLite using Python's `sqlite3` module.
2. Load data from specified CSV files into the SQLite database.
3. Retrieve and display the names of tables within the database.
4. Fetch and present the schema information for each table to understand the data structure.
5. Execute custom SQL queries and return the results for further processing or analysis.

The notebook utilizes a configuration file, `settings.ini`, to manage database paths and data source URLs, promoting a modular and easily configurable code environment.

### Outcomes

- **Database Connection**: A reusable database connection function was established, allowing for easy connection handling throughout the notebook.
- **Data Loading**: Data from external CSV files was loaded into the SQLite database, creating the `high_schools` and `sat_records` tables.
- **Table Inspection**: Functions were created to list all tables in the database and to retrieve detailed schema information for each table, aiding in data understanding and validation.
- **Query Execution**: A generic function to run SQL queries was implemented, providing a foundation for executing data retrieval and manipulation commands.


In [1]:
import sqlite3
import pandas as pd
import configparser

In [2]:
def db_connection(db_path:str):
    """
    Creates a connection and a cursor to the SQLite database.

    Parameters:
    db_path (str): Path to the SQLite database file.

    Returns:
    tuple: A tuple containing the connection and cursor to the database.
    """
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    return conn, cursor

def load_data_to_sqlite(db_path: str, data_url: str, table_name: str) -> None:
    """
    Loads data from a CSV URL into an SQLite table.
    """
    conn, cursor = db_connection(db_path)
    df = pd.read_csv(data_url)
    df.to_sql(table_name, conn, index=False, if_exists='replace')
    conn.close()
        
def get_table_names(db_path: str) -> list[type]:
    """
    Retrieves a list of all table names in the SQLite database.
    """
    conn, cursor = db_connection(db_path)
    cursor.execute('SELECT name from sqlite_master where type="table"')
    tables = cursor.fetchall()
    conn.close()
    return tables      

def get_table_info(db_path:str, table_name: str) -> list[type]:
    """
    Retrieves the schema information of a given table.
    """
    conn, cursor = db_connection(db_path)
    cursor.execute(f'PRAGMA table_info({table_name})')
    info = cursor.fetchall()
    conn.close()
    return info

def run_query(db_path: str, statement: str) -> list[type]:
    """
    Executes a SQL statement and returns the results.
    """
    conn, cursor = db_connection(db_path)
    cursor.execute(statement)
    results = cursor.fetchall()
    conn.close()
    return results

In [3]:
# Constants
db_path = 'schools.db'
highschools_url = "https://raw.githubusercontent.com/sql-fundamentals-jigsaw/mod-1-sql-curriculum/master/2-sql-relations/2-belongs-to-hs/data/highschools.csv"
sat_records_url = "https://raw.githubusercontent.com/sql-fundamentals-jigsaw/mod-1-sql-curriculum/master/2-sql-relations/2-belongs-to-hs/data/sat_records.csv"

# load data in sqlite3 - 'schools.db'
load_data_to_sqlite(db_path, highschools_url, 'high_schools')
load_data_to_sqlite(db_path, sat_records_url, 'sat_records') 

In [4]:
get_table_names(db_path)

[('high_schools',), ('sat_records',)]

In [5]:
 get_table_info(db_path, table_name ='high_schools')

[(0, 'id', 'INTEGER', 0, None, 0),
 (1, 'dbn', 'TEXT', 0, None, 0),
 (2, 'school_name', 'TEXT', 0, None, 0),
 (3, 'boro', 'TEXT', 0, None, 0),
 (4, 'total_students', 'INTEGER', 0, None, 0),
 (5, 'graduation_rate', 'REAL', 0, None, 0),
 (6, 'attendance_rate', 'REAL', 0, None, 0),
 (7, 'college_career_rate', 'REAL', 0, None, 0)]

In [89]:
statement ="""
SELECT DISTINCT boro FROM high_schools LIMIT 10
"""
run_query(db_path, statement)

[('K',), ('M',), ('Q',), ('X',), ('R',)]

In [92]:
list_of_boro = run_query(db_path, statement)
[boro[0] for boro in list_of_boro ]

['K', 'M', 'Q', 'X', 'R']

Each of the following questions require information in both of the tables in the database. Use the JOIN clause to find the solution.

In [7]:
# In which boro is the school that has the highest writing score?

statement = """
SELECT 
  high_schools.boro,
  max(sat_records.writing_score)
FROM 
  high_schools
JOIN sat_records 
    ON high_schools.dbn = sat_records.dbn
GROUP BY 
  high_schools.boro
ORDER BY 
  max(writing_score) desc
LIMIT 1
"""

run_query(db_path, statement)

[('M', 682.0)]

In [8]:
# In which boro is the school with the lowest math average?

statement = """
SELECT 
  high_schools.boro,
  min(sat_records.math_avg)
FROM 
  high_schools
JOIN sat_records 
    ON high_schools.dbn = sat_records.dbn
GROUP BY 
  high_schools.boro
ORDER BY 
  min(sat_records.math_avg)
LIMIT 1
"""
run_query(db_path, statement)

[('X', 312.0)]

In [9]:
# What is the highest math_avg for schools with more than 1000 students?

statement = """
SELECT 
  high_schools.dbn, 
  max(sat_records.math_avg) 
FROM 
    sat_records
JOIN high_schools 
    on sat_records.dbn = high_schools.dbn
GROUP BY
    high_schools.dbn
HAVING 
    total_students > 1000
ORDER BY 
    max(sat_records.math_avg) desc
LIMIT 1
"""

run_query(db_path, statement)

[('02M475', 735.0)]

In [10]:
# What is the average number of test takers in each boro?

statement = """
select 
  high_schools.boro, 
  avg(sat_records.num_test_takers) 
from sat_records
join high_schools 
  on sat_records.dbn = high_schools.dbn
group by 
  high_schools.boro
order by 
  max(sat_records.num_test_takers) desc
"""

run_query(db_path, statement)

[('K', 126.33673469387755),
 ('Q', 199.51666666666668),
 ('M', 110.34177215189874),
 ('R', 300.5),
 ('X', 80.3875)]

In [11]:
# What is the attendance rate of schools with math_avg greater than 500? Order your results by the attendance rate (descending) limit to the first five results

statement = """
SELECT 
  high_schools.attendance_rate
FROM sat_records
JOIN high_schools 
  on sat_records.dbn = high_schools.dbn
WHERE math_avg > 500 
ORDER BY 
  high_schools.attendance_rate desc
LIMIT 5
"""

run_query(db_path, statement)

[(0.98,), (0.97,), (0.97,), (0.97,), (0.97,)]

In [12]:
# What is the graduation rate of schools with a math_avg less than 500? Order your results by the graduation rate (ascending) limit to ten results

statement = """
select 
  high_schools.graduation_rate
from sat_records
join high_schools 
  on sat_records.dbn = high_schools.dbn
where math_avg < 500 
order by 
  high_schools.graduation_rate
limit 10
"""

run_query(db_path, statement)

[(None,),
 (None,),
 (None,),
 (None,),
 (0.39,),
 (0.46,),
 (0.47,),
 (0.49,),
 (0.5,),
 (0.5,)]

In [13]:
# For schools with a math avg greater than 500, what is the average graduation rate?

statement = """
select 
  avg(high_schools.graduation_rate)
from sat_records
join high_schools 
  on sat_records.dbn = high_schools.dbn
where math_avg > 500
"""

run_query(db_path, statement)

[(0.9769999999999999,)]

In [14]:
# What is the total number of test takers in each boro?

statement = """
select 
  high_schools.boro, 
  sum(sat_records.num_test_takers) 
from sat_records
join high_schools 
  on sat_records.dbn = high_schools.dbn
group by 
  high_schools.boro
"""

run_query(db_path, statement)

[('K', 12381.0), ('M', 8717.0), ('Q', 11971.0), ('R', 3005.0), ('X', 6431.0)]

In [15]:
# What is the average combined reading and math scores for each boro?

statement = """
select 
  boro,
  avg(sat_records.reading_avg + sat_records.math_avg) 
from sat_records
join high_schools 
  on sat_records.dbn = high_schools.dbn
Group by 
  boro
"""
run_query(db_path, statement)

[('K', 795.2857142857143),
 ('M', 869.5822784810126),
 ('Q', 874.5666666666667),
 ('R', 930.0),
 ('X', 778.2375)]

In [16]:
# Find the top five schools that have the largest differences between num_test_takers and total_students

statement = """
select 
  name,
  max(high_schools.total_students - sat_records.num_test_takers ) 
from sat_records
join high_schools 
  on sat_records.dbn = high_schools.dbn
Group by 
  sat_records.dbn
order by 
  max(high_schools.total_students - sat_records.num_test_takers ) desc
limit 5
"""
run_query(db_path, statement)

[('BROOKLYN TECHNICAL HIGH SCHOOL', 4561.0),
 ('FORT HAMILTON HIGH SCHOOL', 3888.0),
 ('FRANCIS LEWIS HIGH SCHOOL', 3623.0),
 ('MIDWOOD HIGH SCHOOL', 3234.0),
 ('JAMES MADISON HIGH SCHOOL', 3139.0)]

In [17]:
statement ="""
SELECT
  hs.boro,
  SUM(hs.total_students) - SUM(sr.num_test_takers) AS difference
FROM 
  high_schools hs
JOIN 
  sat_records sr ON hs.dbn = sr.dbn
GROUP BY 
  hs.boro
ORDER BY 
  difference ASC;
"""

run_query(db_path, statement)

[('R', 15627.0),
 ('X', 39112.0),
 ('M', 45460.0),
 ('Q', 61834.0),
 ('K', 64609.0)]

# Codes ready for dev and prod
### Write the functions in a class

In [68]:
import sqlite3
import pandas as pd
from functools import wraps

def check_connection(func):
    """Decorator to check if the database connection is established."""
    @wraps(func)
    def wrapper(self, *args, **kwargs):
        if self.db_connect():
            return func(self, *args, **kwargs)
        else:
            raise ConnectionError("Failed to connect to the database.")
    return wrapper

class SchoolDatabase:
    def __init__(self, db_path: str):
        self.db_path = db_path
        
    def db_connect(self):
        """
        Creates a connection to the SQLite database.

        Parameters:
        db_path (str): Path to the SQLite database file.

        Returns:
        sqlite3.Connection: A connection object to the database.
        """
        try:
            conn = sqlite3.connect(self.db_path)
            print("Database connection successful.")
            return conn
        except sqlite3.Error as e:
            print(f'Database connection failed: {e}')
            return None
    
    def load_data_to_sqlite(self, data_url: str, table_name: str) -> None:
        """
        Loads data from a CSV URL into an SQLite table.

        Parameters:
        db_path (str): Path to the SQLite database file.
        data_url (str): URL to the CSV data.
        table_name (str): Name of the table to create or replace in the SQLite database.
        """
        conn = None
        try:
            conn = self.db_connect()
            df = pd.read_csv(data_url)
            df.to_sql(table_name, conn, index=False, if_exists='replace')
            print(f"Data loaded successfully into '{table_name}' table.")
        except sqlite3.Error as e:
            print(f"An error occurred while loading data into '{table_name}': {e}")
        finally:
            if conn:
                conn.close()
        
    def get_table_names(self) -> list:
        """
        Retrieves a list of all table names in the SQLite database.

        Parameters:
        db_path (str): Path to the SQLite database file.

        Returns:
        list: A list of table names.
        """
        conn = None
        try:
            conn = self.db_connect()
            cursor = conn.cursor()
            cursor.execute('SELECT name from sqlite_master where type="table"')
            tables = cursor.fetchall()
            print("Table names retrieved successfully")
            return tables
        except sqlite3.Error as e:
            print(f"Failed to retrieve table names: {e}")
        finally:
            if conn:
                conn.close()
    
    def get_table_info(self, table_name: str) -> list:
        """
        Retrieves the schema information of a given table.

        Parameters:
        db_path (str): Path to the SQLite database file.
        table_name (str): Name of the table to retrieve information for.

        Returns:
        list: A list of tuples containing the schema information.
        """

        conn = None
        try:
            conn = self.db_connect()
            cursor = conn.cursor()
            cursor.execute(f'PRAGMA table_info({table_name})')
            info = cursor.fetchall()
            print(f"Table info for '{table_name}' retrieved successfully.")
            return info
        except sqlite3.Error as e:
            print(f"An error occurred while retrieving info for '{table_name}': {e}")
        finally:
            if conn:
                conn.close()
                
    def run_query(self, statement: str) -> list:
        """
        Executes a SQL statement and returns the results.

        Parameters:
        db_path (str): Path to the SQLite database file.
        statement (str): SQL statement to execute.

        Returns:
        list: A list of tuples containing the query results.
        """
        conn = None
        try:
            conn = self.db_connect()
            cursor = conn.cursor()
            cursor.execute(statement)
            results = cursor.fetchall()
            print("Query executed successfully.")
            return results
        except sqlite3.Error as e:
            print(f"An error occurred while executing the query: {e}")
        finally:
            if conn:
                conn.close()

    def get_difference_between_students_and_test_takers(self) -> list[type]:
        """Calculates the difference between total students and number of test takers for each boro."""
        statement = """
        SELECT 
          hs.boro,
          SUM(hs.total_students) - SUM(sr.num_test_takers) AS difference
        FROM 
          high_schools hs
        JOIN 
          sat_records sr ON hs.dbn = sr.dbn
        GROUP BY 
          hs.boro
        ORDER BY 
          difference ASC;
        """
        try:
            results = self.run_query(statement)
            print("Successfully retrieved the difference between total students and number of test takers for each boro.")
            return results
        except Exception as e:
            print(f"An error occurred while calculating the difference: {e}")
            return []

In [69]:
# Read settings from the configuration file
config = configparser.ConfigParser()
config.read('../../config/settings.ini')

# # Constants from the configuration file
db_path = config['database']['db_path']
highschools_url = config['csv_data_sources']['highschools_url']
sat_records_url = config['csv_data_sources']['sat_records_url']

# Usage
db = SchoolDatabase(db_path)
db.load_data_to_sqlite(highschools_url, 'high_schools')
db.load_data_to_sqlite(sat_records_url, 'sat_records')

Database connection successful.
Data loaded successfully into 'high_schools' table.
Database connection successful.
Data loaded successfully into 'sat_records' table.


In [70]:
print(dir(db))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'db_connect', 'db_path', 'get_difference_between_students_and_test_takers', 'get_table_info', 'get_table_names', 'load_data_to_sqlite', 'run_query']


In [72]:
db.get_table_info('high_schools')
#db.get_table_names()

Database connection successful.
Table info for 'high_schools' retrieved successfully.


[(0, 'id', 'INTEGER', 0, None, 0),
 (1, 'dbn', 'TEXT', 0, None, 0),
 (2, 'school_name', 'TEXT', 0, None, 0),
 (3, 'boro', 'TEXT', 0, None, 0),
 (4, 'total_students', 'INTEGER', 0, None, 0),
 (5, 'graduation_rate', 'REAL', 0, None, 0),
 (6, 'attendance_rate', 'REAL', 0, None, 0),
 (7, 'college_career_rate', 'REAL', 0, None, 0)]

In [50]:
# Find the top five schools that have the largest differences between num_test_takers and total_students

statement = """
SELECT 
  name,
  max(high_schools.total_students - sat_records.num_test_takers ) 
FROM sat_records
JOIN high_schools 
  on sat_records.dbn = high_schools.dbn
GROUP BY 
  sat_records.dbn
ORDER BY 
  max(high_schools.total_students - sat_records.num_test_takers ) desc
LIMIT 5
"""
db.run_query(statement)

Database connection successful.
Query executed successfully.


[('BROOKLYN TECHNICAL HIGH SCHOOL', 4561.0),
 ('FORT HAMILTON HIGH SCHOOL', 3888.0),
 ('FRANCIS LEWIS HIGH SCHOOL', 3623.0),
 ('MIDWOOD HIGH SCHOOL', 3234.0),
 ('JAMES MADISON HIGH SCHOOL', 3139.0)]

In [57]:
records = db.run_query(statement)
new_records = []
for schoolname, diff_num in records:
    if diff_num >= 3500:
        new_records.append({schoolname: diff_num})
    
new_records

[{'BROOKLYN TECHNICAL HIGH SCHOOL': 4561.0},
 {'FORT HAMILTON HIGH SCHOOL': 3888.0},
 {'FRANCIS LEWIS HIGH SCHOOL': 3623.0}]

In [88]:
statement = """
SELECT 
    COUNT(*)
FROM sat_records
"""
db.run_query(statement)[0][0]

Database connection successful.
Query executed successfully.


478